In [1]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from lightgbm.callback import early_stopping, log_evaluation

import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model/other/default/1/model.pkl
/kaggle/input/keplercompleteddata/clean_kepler.csv
/kaggle/input/ktestdata/k2_clean.csv
/kaggle/input/ramidata/df_merged.csv
/kaggle/input/kepler/cumulative_2025.10.03_23.57.10.csv


In [37]:
df = df.drop(["koi_disposition", "koi_pdisposition", "koi_score", "kepid", "kepoi_name", "kepler_name"], axis=1, errors="ignore")


NameError: name 'df' is not defined

In [ ]:
df = pd.read_csv("/kaggle/input/keplercompleteddata/clean_kepler.csv")
df["label"] = df["koi_disposition"].apply(lambda x: 1 if x == "CONFIRMED" else 0)


In [ ]:
X = df.drop("label", axis=1)
y = df["label"]

# stratify — чтобы сохранить баланс классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [ ]:
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1
}

In [ ]:
model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=[train_data, test_data],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=100)
    ],
    num_boost_round=1000
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = (model.predict(X_test) > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
"koi_score" in X_train.columns

In [ ]:
sample = X_test.iloc[4]  
prediction = model.predict([sample])
prediction

In [32]:
import joblib

# Сохраняем модель
joblib.dump(model, 'model.pkl')

NameError: name 'model' is not defined

In [31]:
import numpy as np

# 🔹 Предсказания вероятностей
y_pred_proba = model.predict(X_test)

# 🔹 Преобразуем в DataFrame
results = pd.DataFrame({
    'id': np.arange(len(y_pred_proba)),        # создаём id для каждой строки
    'procent': y_pred_proba,                   # вероятность "Planet"
    'xai': (y_pred_proba > 0.5).astype(int)    # 1 если > 0.5, иначе 0
})

# 🔹 Фильтруем по порогу
results_filtered = results[results['procent'] > 0.5]

# 🔹 Выводим первые 50 строк
print(results_filtered.head(50))

# 🔹 Сохраняем файл для отправки
results_filtered.to_csv('submission.csv', index=False)

NameError: name 'model' is not defined